# PyGML: a python API to the GMQL system

The user imports the library in his python environment

In [1]:
import gmql as gl

Sets the input path from which to take the dataset and the output path where to materialize the results

In [2]:
# loading of a dataset
data_path = "/home/luca/Documenti/resources/hg_narrowPeaks"
output_path = "/home/luca/Documenti/resources/result"

He chooses a parser to use with the data. The available parsers are the one already provided by the GMQL system.
For now there is no possibility for the user to define its own parser due to the fact that the loading of data is done by the GMQL system.

In [3]:
parser = gl.parsers.NarrowPeakParser()

data = gl.GMQLDataset(parser=parser)\
                .load_from_path(path=data_path)
    
data.show_info()

2017-04-16 21:10:45,409 - PyGML logger - INFO - Loading meta data from path /home/luca/Documenti/resources/hg_narrowPeaks


100%|██████████| 2000/2000 [00:17<00:00, 112.87it/s]


2017-04-16 21:11:04,358 - PyGML logger - INFO - dataframe construction


100%|██████████| 115/115 [00:39<00:00,  2.93it/s]

GMQLDataset
	Parser:	NarrowPeakParser
	Index:	0


When the user loads the file, in reality nothing really happens. All the action but the `materialize` are to be intended as an *intention* of action. All the computation is started when a `materialize` is performed.
The only thing that it is really done by the `load_from_path` function is to load all the metadata in memory (in a Pandas dataframe) in order for the user to explore them.

In [15]:
# visualize the metadata
# for visualization purposes we only show the first rows and only 4 columns
data.meta.head()[['ID','antibody','antibody_lab','cell']]

,ID,antibody,antibody_lab,cell
id_sample,,,,
-9219482592272593970,[1362],[STAT3],[Snyder],[MCF10A-Er-Src]
-9203625831147867011,[1960],[CTCF],"[Myers,]",[HepG2]
-9198631368136544855,[1624],[],[],[MCF-7]
-9195412399933906641,[71],[],[],[HGF]
-9172267737713816558,[355],[Pol2],"[Farnham,]",[K562]


This is a demonstration of how the user can create a selection predicate. 
It was the result of a lot of reasoning and hacking of python function overloading and I am pretty sure that is the best trade-off between usability, easy of writing and expressive power.
The user can express arbitrarly complex condition using the usual logical symbols.

In [5]:
# selects all the data coming from cell K562 and having antibody H3K4me3
condition = (gl.MetaField("cell") == 'K562') & (gl.MetaField("antibody") == 'H3K4me3')

filtered_data = data.meta_select(predicate=condition) # test

Here we materialize our results to the `output_path`, and automatically we load in memory the result into two Pandas dataframe for local processing, one for regions and one for metadata. The function directly call the GMQL system and makes it perform all operations and optimizations.

In [6]:
filtered_data = filtered_data.materialize(output_path=output_path)

2017-04-16 21:13:40,197 - PyGML logger - INFO - Loading meta data from path /home/luca/Documenti/resources/result/exp/


100%|██████████| 12/12 [00:00<00:00, 1016.88it/s]

2017-04-16 21:13:40,261 - PyGML logger - INFO - dataframe construction



100%|██████████| 94/94 [00:00<00:00, 135.47it/s]

2017-04-16 21:13:41,002 - PyGML logger - INFO - Loading region data from path /home/luca/Documenti/resources/result/exp/



100%|██████████| 14/14 [00:10<00:00,  1.24it/s]


Let's check that the result is coherent with the performed query

In [16]:
filtered_data.meta.head()[['cell', 'antibody']]

,cell,antibody
id_sample,,
-6509382303831254894,[K562],[H3K4me3]
-5512387921408838373,[K562],[H3K4me3]
-4037276753222902964,[K562],[H3K4me3]
-2728658603483102692,[K562],[H3K4me3]
-2321017725132867669,[K562],[H3K4me3]


In [17]:
filtered_data.regs.head()

,chr,name,pValue,peak,qValue,score,signalValue,start,stop,strand
id_sample,,,,,,,,,,
-494029087713947133,chr1,.,5.18028,-1.0,-1.0,0.0,6.0,118520,118670,*
-494029087713947133,chr1,.,71.79020,-1.0,-1.0,0.0,46.0,137820,137970,*
-494029087713947133,chr1,.,21.24010,-1.0,-1.0,0.0,31.0,138400,138550,*
-494029087713947133,chr1,.,21.24010,-1.0,-1.0,0.0,25.0,138640,138790,*
-494029087713947133,chr1,.,41.58580,-1.0,-1.0,0.0,41.0,138920,139070,*
